In [82]:
import pygraphviz as pgv
import xmltodict as xtd
import requests
from collections import OrderedDict

In [83]:
# Read the RFC Index as XML and convert to a python Dict
#rfcIndexUrl = 'https://www.rfc-editor.org/in-notes/rfc-index.xml'
#xmlData = requests.get(rfcIndexUrl)
#dictData = xtd.parse(xmlData.text)
with open('rfc-index.xml','r') as xmlData:
    dictData = xtd.parse(xmlData.read())
rfcIndex = dictData['rfc-index']

In [84]:
# The RFC Index has the following elements of interest:
rfcIndex.keys()

odict_keys(['@xmlns', '@xmlns:xsi', '@xsi:schemaLocation', 'bcp-entry', 'fyi-entry', 'rfc-entry', 'rfc-not-issued-entry', 'std-entry'])

In [85]:
# Import color definitions to be used in coloring edges and nodes.
with open('svg-colors.txt','r') as colorFile:
    colors = colorFile.readlines()
    colors = [ color.strip() for color in colors]

In [86]:
# Initialize a new digraph, "G", to represent BCPs and RFCs as nodes;
# is-also's, obsolescence, and updates as edges.
G = pgv.AGraph(directed=True)
G.graph_attr['label']="RFC Directed Graph"

In [18]:
# For each element in the BCP Entry list, add it to the graph, and build
# edges for is-also's to RFC nodes.
for bcp in rfcIndex['bcp-entry']:
    G.add_node(bcp['doc-id'])
    try:
        if isinstance(bcp['is-also'], OrderedDict):
            if isinstance(bcp['is-also']['doc-id'],str):
                #print(bcp['is-also']['doc-id'])
                G.add_edge(bcp['doc-id'],bcp['is-also']['doc-id'],label='is-also')
            elif isinstance(bcp['is-also']['doc-id'],list):
                #print(bcp['is-also']['doc-id'])
                for rfc in bcp['is-also']['doc-id']:
                    #print(rfc)
                    G.add_edge(bcp['doc-id'],rfc,label='is-also')
            else:
                print("error: is-also has unexpected format")
    except KeyError:
        pass

In [19]:
# For each element in the FYI Entry list, add it to the graph, and build
# edges for is-also's to RFC nodes.
for fyi in rfcIndex['fyi-entry']:
    G.add_node(fyi['doc-id'])
    try:
        if isinstance(fyi['is-also'], OrderedDict):
            if isinstance(fyi['is-also']['doc-id'],str):
                G.add_edge(fyi['doc-id'],fyi['is-also']['doc-id'],label='is-also')
            elif isinstance(fyi['is-also']['doc-id'],list):
                for rfc in fyi['is-also']['doc-id']:
                    G.add_edge(fyi['doc-id'],rfc,label='is-also')
            else:
                print("error: is-also has unexpected format")
    except KeyError:
        pass

In [89]:
# For each element in the RFC Entry list, add it to the graph, and build
# edges for is-also's, updates, obsoletes.
counter = len([rfc for rfc in rfcIndex['rfc-entry']])
print(counter)

for rfc in rfcIndex['rfc-entry'][:counter]:
    G.add_node(rfc['doc-id'])

#G.add_node(rfcIndex['rfc-entry'][0]['doc-id'])

#for rfc in rfcIndex['rfc-entry'][:counter]:
#    G.add_node(rfc['doc-id'])

#for rfc in rfcIndex['rfc-entry']:
#    try:
#        G.add_node(rfc['doc-id'])
#    except:
#        print("error reached")
#    try:
#        if isinstance(rfc['is-also'], OrderedDict):
#            if isinstance(rfc['is-also']['doc-id'],str):
#                G.add_edge(rfc['doc-id'],rfc['is-also']['doc-id'],label='is-also')
#            elif isinstance(rfc['is-also']['doc-id'],list):
#                for standard in rfc['is-also']['doc-id']:
#                    G.add_edge(rfc['doc-id'],standard,label='is-also')
#            else:
#                print("error: is-also has unexpected format")
#    except Exception as e:
#        pass
#obsoletes', OrderedDict([('doc-id', 'RFC0003')])),
#              ('obsoleted-by', OrderedDict([('doc-id', 'RFC0016')])),
#              ('updated-by',

8477


In [16]:
#[rfc['doc-id'] for rfc in rfcIndex['rfc-entry']]


In [90]:
# Generate the graph.
# N.B. As of this writing, this takes several minutes run.
G.layout()
G.write('rfc-visualization.dot')
G.draw('rfc-visualization.svg')